In [1]:
import joblib
import dice_ml
import shap
import lime
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from operator import itemgetter
from sklearn.model_selection import train_test_split, cross_val_score
from explainerdashboard import ClassifierExplainer
from sklearn.metrics import ConfusionMatrixDisplay, precision_recall_fscore_support, classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE
from alibi.explainers.ale import ALE, plot_ale

In [29]:
class IREX:
    def __init__(self):
        ## The names of the explanation methods used are defined.
        self.X = None
        self.y = None
        self.df = None
        self.label = None
        self.train_cols = None
        self.dataset_file = None
        self.target_names = None
        self.name_methods = ["Importance", "LIME", "SHAP", "ALE"]

    def load_dataset_from_csv(self, dataset_csv, target_names, dependant_variables_range=[0,-1], target_variable=[-1]):
        self.dataset_file = dataset_csv
        self.df = pd.read_csv(dataset_csv)
        self.train_cols = self.df.iloc[:, dependant_variables_range[0]:dependant_variables_range[1]].columns
        print(self.train_cols)
        self.label = self.df.iloc[:, target_variable].columns
        print(self.label)
        self.X = self.df[self.train_cols]
        self.y = self.df[self.label]
        self.target_names = target_names
        print(self.df)

    # def load_dataset(self, train_cols_dataframe, target_col_dataframe, target_names):
    #     self.train_cols = train_cols_dataframe
    #     self.label = target_col_dataframe
    #     self.X = df [self.train_cols]
    #     self.y = df [self.label]
    #     self.target_names = target_names
    #
    # def load_expected_answers(csv_filename, item_column_index = 0, class_column_index = 1, expected_answer_index = 2):
    #     #cargar scv y crear una función que devuelve el valor esperado dado el índice del item y la clase:
    #     # el csv tiene: col0: item index, col1: class, col2: valoresperado
    #     # def eaf(item,class):
    #     self.eaf = eaf
    #
    # def set_expected_answers_function(function):
    #     self.eaf = function
    #
    #
    # def reset_IREX(self):
    #     ## Generation of the question status list automatically.
    #     question = list(range(1, self.train_cols.shape[1]))
    #     status = ['In use' for _ in range(1, self.train_cols.shape[1])]
    #     self.ds = pd.DataFrame({'Question': question, 'Status': status})
    #
    #     ## Save the changed statuses in the dataset.
    #     self.ds.to_csv(path_dataset_qs, index = False)
    #
    #     ## Generation of the data structure by iteration automatically.
    #     columns = ['Question', 'Acurracy global']
    #
    #     for index in range(0, len(name_class)):
    #         columns.append('Precision_' + str(index))
    #         columns.append('Recall_' + str(index))
    #         columns.append('F1_score_' + str(index))
    #         columns.append('Support_' + str(index))
    #
    #     self.dd = pd.DataFrame(columns = columns)
    #
    #     ## Save the changed data in the dataset.
    #     self.dd.to_csv(path_dataset_qd, index = False)
    #
    #     ## The number of iterations performed in the explanation is defined.
    #     self.iteration = 0
    #
    # def oversample():
    #     ## The oversample function used is defined.
    #     random_state = 13
    #     oversample = SMOTE(random_state = random_state)
    #     self.X, self.y = oversample.fit_resample(self.X, self.y)
    #
    # def train_model(do_oversample=true, test_size=0.33, hidden_layer_sizes = (10,), activation = 'logistic', solver = 'adam', alpha = 1,
    #          learning_rate = 'constant', learning_rate_init = 0.0001, max_iter = 100000, random_state = 77):
    #     if(do_oversample):
    #         oversamplle()
    #     self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size, random_state=seed)
    #     ## The value of the alpha parameter is kept at 1 as well as the value of the seed of random_state at 77.
    #     self.mlp = MLPClassifier(hidden_layer_sizes, activation, solver, alpha,
    #          learning_rate , learning_rate_init, max_iter0, random_state)
    #
    #     ## We print the data of the final generated model.
    #     self.mlp.fit(self.X_train, self.y_train)
    #
    # def dump_model(filename):
    #     joblib.dump(self.mlp, filename)
    #
    # def load_model(filename):
    #     self.mlp = return joblib.load(filename)
    #
    # def evaluate_model():
    #     ## The data for the generation of the confession matrix are defined.
    #     confusion_matrix = ConfusionMatrixDisplay.from_estimator(self.mlp, self.X_test, self.y_test, display_labels = target_names,
    #                                                      cmap = plt.cm.Blues)
    #     confusion_matrix.ax_.set_title("Confusion Matrix")
    #     plt.show()
    #
    #     ## The necessary detailed prediction is generated.
    #     y_pred = mlp.predict(self.X_test)
    #
    #     ## The detailed statistics of the model are printed.
    #     print(classification_report(self.y_test,y_pred))
    #
    # def compute_explanation(PAI_significance_thresold = .1):
    #     self.iteration++
    #     __run_ALE()
    #     __run_LIME()
    #     __run_SHAP()
    #     __run_Feature_Importance()
    #     __compute_PAI(PAI_significance_thresold, selection_method = 1)#no liarse con esto. Hacer el más sencillo
    #
    # def __run_ALE():
    #     #rellenar
    #     #In analysis of the model through the results obtained by ALE, it is possible to identify the behavior
    #     #of the neural network.
    #     ## The necessary parameters for the use of ALE are established according to the model.
    #     proba_fun_lr = mlp.predict_proba
    #     proba_ale_lr = ALE(proba_fun_lr, feature_names = train_cols, target_names = target_names)
    #     proba_exp_lr = proba_ale_lr.explain(X_train.to_numpy())
    #     plot_ale(proba_exp_lr, n_cols=2, features=list(range(df.shape[1]-1)),fig_kw={'figwidth': 10, 'figheight': 180});
    #
    # def __run_LIME():
    #     #rellenar
    #     #We proceed to explain the improved model using LIME for the Low, Medium and High classes.
    #     explainer = lime.lime_tabular.LimeTabularExplainer(X_train.to_numpy(), categorical_features=train_cols,
    #                                                feature_names=train_cols, class_names=target_names,
    #                                                discretize_continuous=True)
    #     #The first individual present in the test data is explained.
    #     exp = explainer.explain_instance(X_test.to_numpy()[0], mlp.predict_proba, num_features=6,top_labels=1)
    #     exp.show_in_notebook(show_table=True, show_all=False)
    #     #The second individual present in the test data is explained.
    #     exp = explainer.explain_instance(X_test.to_numpy()[1], mlp.predict_proba, num_features=6,top_labels=1)
    #     exp.show_in_notebook(show_table=True, show_all=False)
    #     #The last individual present in the test data is explained.
    #     exp = explainer.explain_instance(X_test.to_numpy()[-1], mlp.predict_proba, num_features=6,top_labels=1)
    #     exp.show_in_notebook(show_table=True, show_all=False)
    # def __run_SHAP():
    #     #rellenar
    #     #SHAP explanatory values are displayed according to the class to which the selected individual belongs according to the prediction of the model.
    #     #The first individual present in the test data is explained.
    #     ## Individual Selection.
    #     individual = X_test.iloc[[0]]
    #
    #     ## Individual explanation by SHAP.
    #     explainer = shap.KernelExplainer(mlp.predict_proba, X_train,feature_names = train_cols, output_names = target_names)
    #     shap_values = explainer.shap_values(individual)
    #     clase = mlp.predict(individual)[0]
    #     shap.summary_plot(shap_values[clase], individual, feature_names = train_cols, plot_type = "bar")
    #     #The second individual present in the data test is explained.
    #     ## Individual Selection.
    #     individual = X_test.iloc[[1]]
    #
    #     ## Individual explanation by SHAP.
    #     explainer = shap.KernelExplainer(mlp.predict_proba, X_train,feature_names = train_cols, output_names = target_names)
    #     shap_values = explainer.shap_values(individual)
    #     clase = mlp.predict(individual)[0]
    #     shap.summary_plot(shap_values[clase], individual, feature_names = train_cols, plot_type = "bar")
    #     #The last individual present in the data test is explained.
    #     ## Individual Selection.
    #     individual = X_test.iloc[[-1]]
    #
    #     ## Individual explanation by SHAP.
    #     explainer = shap.KernelExplainer(mlp.predict_proba, X_train,feature_names = train_cols, output_names = target_names)
    #     shap_values = explainer.shap_values(individual)
    #     clase = mlp.predict(individual)[0]
    #     shap.summary_plot(shap_values[clase], individual, feature_names = train_cols, plot_type = "bar")
    # #def __run_Feature_Importance():
    #     #rellenar
    #
    # #def __compute_PAI(selection_method = 1):
    #     #rellenar
    #     #según el número que pongan, es el método para determinar las PAIs (los 5 casos que nos dió juan)
    #
    # def visualize_global_heatmap(_class):
    #     #según la clase, se tiene que visualizar el mapa de calor
    #
    # def visualize_class_heatmap(_class, XAI_method):
    #     #según la clase que se elija y el método, se desplegará el mapa de calor solo de dicha clase
    #
    # def visualize_average_heatmap(XAI_method):
    #     #
    #
    # def visualize_detailled_heatmap(XAI_method):
    #     #
    #
    # def refine_dataset(anomalous_list):
    #     # remove items from dataframes self.X self.y
    #
    #
    # def plot_global_process(accuracy = true, precision = true, recall = true):
    #     #todo
    #     #depende de lo que elijan, que se despliegue el dato (aunque creo que con este codig no se puede)
    #     #We get the detailed statistics of the final model.
    #     ## The necessary detailed prediction is generated.
    #     y_pred = mlp.predict(X_test)
    #
    #     ## The detailed statistics of the model are printed.
    #     print('Classification accuracy =',accuracy_score(y_test,y_pred) * 100,'%\n')
    #     print(classification_report(y_test,y_pred))

In [30]:
irex = IREX()
irex.load_dataset_from_csv('Dataset_MO_ENG.csv', ["Low","Mid","High"], [0,102], [-1])
# irex.load_expected_answers('answers.csv')
# irex.reset()
# irex.oversample()
# irex.train()
# irex.evaluate_model()
# irex.compute_explanation()
# irex.visualize_global_heatmap("High")
# irex.visualize_class_heatmap("High","LIME")
# irex.refine_dataset([2,7,9,3,13,34])
#
# irex.train()
# irex.evaluate_model()
# irex.compute_explanation()
# irex.visualize_global_heatmap("High")
# irex.visualize_class_heatmap("High","LIME")
# irex.refine_dataset([2,7,9,3,13,34])

Index(['1. Most of the time I have difficulty concentrating on simple tasks',
       '2. I don't feel like doing my daily duties',
       '3. My friends or family have told me that I look different',
       '4. When I think about the future it is difficult for me to imagine it clearly',
       '5. People around me often ask me how I feel',
       '6. I consider that my life is full of good things',
       '7. My hobbies are still important to me',
       '8. I'm still as punctual as I have always been',
       '9. If I had the chance, I would spend all day in my bed',
       '10. I have found that I can spend a lot of time scrolling the screen \nof my cell phone without searching or stopping at anything in particular',
       ...
       '93. People steal because they have needs', '94. I lose control easily',
       '95. Neighbors must put up with each other's noises without complaining',
       '96. Littering on public roads is wrong',
       '97. People who commit crimes have their re